# Tensor Program Abstraction in Action

1. 用 TVM 的 2 种 DSL 写算子。
    - TVMScript 的 TIR 写 矩阵加法
    - Tensor Expression（简称 TE）写矩阵乘法
2. 程序优化程序。tvm scheduler 把矩阵乘的性能提升 ~10 倍。

In [1]:
import tvm
from tvm.ir.module import IRModule

from tvm.script import tir as T
from tvm import te

import numpy as np

print('tvm versin: %s' % tvm.__version__)

tvm versin: 0.13.dev0


## 1. 用 TVMScript 写矩阵加法

### 1.1. 用 TVMScript 实现原始程序

In [2]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def main(A: T.Buffer(128, "float32"), 
             B: T.Buffer(128, "float32"), 
             C: T.Buffer(128, "float32")):
        # extra annotations for the function
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i in range(128):
            with T.block("C"):
                # declare a data parallel iterator on spatial domain
                vi = T.axis.spatial(128, i)
                C[vi] = A[vi] + B[vi]
print(type(MyModule))
MyModule.show()

<class 'tvm.ir.module.IRModule'>


### 1.2. Build and run

1. build 成可执行 module，即，一种动态库。
2. 用 tvm runtime 加载动态库并运行。

In [3]:
rt_mod = tvm.build(MyModule, target="llvm")  # The module for CPU backends.
print('rt_mod type: %s' % type(rt_mod))

# After build, mod contains a collection of runnable functions.
# We can retrieve each function by its name.
func = rt_mod["main"]

a = tvm.nd.array(np.arange(128, dtype="float32"))
b = tvm.nd.array(np.ones(128, dtype="float32")) 
c = tvm.nd.empty((128,), dtype="float32")

func(a, b, c)

print('a: ', a.asnumpy()[:10])
print('b: ', b.asnumpy()[:10])
print('c: ', c.asnumpy()[:10])

rt_mod type: <class 'tvm.driver.build_module.OperatorModule'>
a:  [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
b:  [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
c:  [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


### 1.3. Transform the program

使用程序 (tvm scheduler) 优化程序 (MyModule). schedule 是 TVM 提供的调度源语，通俗的理解，就是优化算法的集合。

In [4]:
sch = tvm.tir.Schedule(MyModule)

# get block to transform(optimize)
block_c = sch.get_block("C")
# get loop in the block to transform(optimize)
(i,) = sch.get_loops(block_c)

# 3 transformation steps: 
# 1. tile, 2. reorder, 3. parallel
# note: split 128 to 8 * 4 * 4
i_0, i_1, i_2 = sch.split(i, factors=[None, 4, 4])
sch.reorder(i_0, i_2, i_1)
sch.parallel(i_0)
sch.mod.show()

Run 优化后的程序，并验证计算结果的正确性不变


In [5]:
c2 = tvm.nd.empty((128,), dtype="float32")

transformed_mod = tvm.build(sch.mod, target="llvm")  # The module for CPU backends.
transformed_mod["main"](a, b, c2)

# check results are the same as before
np.testing.assert_allclose(c.asnumpy(), c2.asnumpy(), rtol=1e-5, atol=1e-5)

## 2. TE (Tensor Expression) 写矩阵乘法

### 2.1. 用 TE 实现原始程序

与 TVMScript 的区别是：
1. TE 抽象的层次更高，使用更简单。
2. TVMScript 更底层，控制能力更强，但，tiling 等细节，也都需要开发者自己实现。

作为对比，先用 TE 实现矩阵加法。很明显，比 TVMScript 简单很多。

In [6]:
# data. input & output 内存分配
A = te.placeholder((128, ), name="A")
B = te.placeholder((128, ), name="B")

# 矩阵加法
C = te.compute((128,), lambda i: A[i] + B[i], name="C")

# 根据 tvm 对设计，生成一个 prim func，以便用 TVM 做优化这个 func。
func = te.create_prim_func([A, B, C])
# function name 标记为 main。也是 TVM 的要求，必须有一个 main 函数作为 IRModule 的入口。
func = func.with_attr("global_symbol", "main")
ir_mod_from_te = IRModule({"main": func})

ir_mod_from_te.show()

用 TE 实现矩阵乘法。并用 TVM build & run。-- 这是优化前的原始程序。

In [7]:
M = 1024
K = 1024
N = 1024

# The default tensor type in tvm
dtype = "float32"

target = "llvm"
dev = tvm.device(target, 0)

# Algorithm
k = te.reduce_axis((0, K), "k")
A = te.placeholder((M, K), name="A")
B = te.placeholder((K, N), name="B")
C = te.compute((M, N), lambda m, n: te.sum(A[m, k] * B[k, n], axis=k), name="C")

# Default schedule
func = te.create_prim_func([A, B, C])
func = func.with_attr("global_symbol", "main")
ir_module = IRModule({"main": func})
ir_module.show()

# build and run
func = tvm.build(ir_module, target="llvm")  # The module for CPU backends.

a = tvm.nd.array(np.random.rand(M, K).astype(dtype), dev)
b = tvm.nd.array(np.random.rand(K, N).astype(dtype), dev)
c = tvm.nd.array(np.zeros((M, N), dtype=dtype), dev)
func(a, b, c)

evaluator = func.time_evaluator(func.entry_name, dev, number=1)
t_baseline = evaluator(a, b, c).mean
print("Baseline: %f" % t_baseline)

Baseline: 2.032805


用 TVM scheduler 优化程序。修改后的 loop，对 cache 更友好。实测，性能提升 10x。

In [8]:
sch = tvm.tir.Schedule(ir_module)
block_c = sch.get_block("C")
# Get loops surronding the block
(y, x, k) = sch.get_loops(block_c)
# step 1: tile (split)
block_size = 32
yo, yi = sch.split(y, [None, block_size])
xo, xi = sch.split(x, [None, block_size])

# step 2: reorder
sch.reorder(yo, xo, k, yi, xi)
sch.mod.show()

# build and run
func = tvm.build(sch.mod, target="llvm")  # The module for CPU backends.

c = tvm.nd.array(np.zeros((M, N), dtype=dtype), dev)
func(a, b, c)

evaluator = func.time_evaluator(func.entry_name, dev, number=1)
t_new = evaluator(a, b, c).mean
print("after transformation: %f. baseline: %f, improved: %.2fx" % (
    t_new, t_baseline, t_baseline/t_new))

after transformation: 0.166623. baseline: 2.032805, improved: 12.20x
